# Match table between VeloGames and Cycling Stats

In [2]:
import pandas as pd

In [3]:
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', -1)  # or 199

In [4]:
cs_results = pd.read_pickle('cs_rider_results.pkl')

In [5]:
vg_riders = pd.read_pickle('vg_rider_list.pkl')

In [6]:
vg_riders.head(2)

,rider,team,class,points,cost
0,Primož Roglič,Team Jumbo-Visma,All Rounder,0,24
1,Simon Yates,Mitchelton-Scott,Climber,0,22


## Fuzzy match cs and vg names

### Clean cs rider names

In [7]:
### Revisit this code!
cs_riders = cs_results[['rider_name']].drop_duplicates().reset_index(drop=True)

rider_names = []
for rider in cs_riders.rider_name.str.strip().str.split():
    rider_name = rider[1:]
    rider_name.append(rider[0].title())
    rider_name = ' '.join(rider_name)
    rider_names.append(rider_name)
    
cs_riders['cleaner_name'] = rider_names

In [8]:
cs_riders.head(2)

,rider_name,cleaner_name
0,LANDA Mikel,Mikel Landa
1,AMADOR Andrey,Andrey Amador


### Match Names

In [9]:
from difflib import SequenceMatcher

In [10]:
matches =[]
for cs in cs_riders.cleaner_name:
    size = []
    for vg in vg_riders.rider:
        size.append(SequenceMatcher(None,cs,vg).ratio())
    match = size.index(max(size))
    guess = list(vg_riders.rider)[match]
            
    match_dict = {
        'cs':cs,
        'vg':guess,
        #'size':max(size) #this is just to quickly eyeball the matches
    }
    matches.append(match_dict)
#pd.DataFrame(matches).sort_values('size') #this is just to quickly eyeball the matches

matches = pd.DataFrame(matches)

In [12]:
map_dict = dict(zip(matches.cs,matches.vg))

In [17]:
cs_riders['vg_rider'] = cs_riders['cleaner_name'].map(map_dict)

In [19]:
cs_riders = cs_riders[['rider_name','vg_rider']]
cs_riders.columns = ['cs_rider','vg_rider']

In [21]:
cs_riders.to_pickle('cs_vg_match.pkl')